In [1]:


pip install tensorflow==2.19.0

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install pandas


Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install scikit-learn


Note: you may need to restart the kernel to use updated packages.


In [8]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import OneHotEncoder

In [9]:
file_path = "final.csv"  
df = pd.read_csv(file_path)

In [10]:
print(df.head())

   period_id  seconds  start_x  start_y   end_x   end_y  action_type  result  \
0          1      0.0   52.500   34.000  43.365  36.312            0       1   
1          1      0.5   43.365   36.312  48.720  34.680           21       1   
2          1      1.0   48.720   34.680  45.150  50.184            0       1   
3          1      4.0   46.200   48.008  39.270  63.376            0       1   
4          1      5.0   39.270   63.376  42.945  63.920           21       1   

   possession_group  
0                 1  
1                 1  
2                 1  
3                 1  
4                 1  


In [11]:
# Encode 'action_type' using One-Hot Encoding
encoder = OneHotEncoder(sparse_output=False)
action_encoded = encoder.fit_transform(df[['action_type']])

In [13]:
# Define input_dim (this must be set after encoding)
input_dim = action_encoded.shape[1]  # Number of unique action types
encoding_dim = 5  # Number of features to learn

In [14]:
# Encoder
input_layer = keras.layers.Input(shape=(input_dim,))
encoded = keras.layers.Dense(encoding_dim, activation="relu")(input_layer)

# Decoder
decoded = keras.layers.Dense(input_dim, activation="softmax")(encoded)

# Autoencoder Model
autoencoder = keras.models.Model(input_layer, decoded)
autoencoder.compile(optimizer="adam", loss="categorical_crossentropy")

In [15]:
# Train the model
autoencoder.fit(action_encoded, action_encoded, epochs=50, batch_size=32, shuffle=True)

Epoch 1/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 2.9218   
Epoch 2/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 2.7456 
Epoch 3/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 2.5457 
Epoch 4/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 2.3317 
Epoch 5/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 2.0816 
Epoch 6/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.8311 
Epoch 7/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.5441 
Epoch 8/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.2785 
Epoch 9/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.1232 
Epoch 10/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.9562 
Epoch 11/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.8886 
Epoch 12/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.7504 
Epoch 13/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.7056 
Epoch 14/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.6635 
Epoch 15/50
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.6235 
Ep

In [16]:
# Extract the encoded features (patterns)
encoder_model = keras.models.Model(input_layer, encoded)
encoded_actions = encoder_model.predict(action_encoded)

41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


In [17]:
# Convert back to DataFrame
encoded_df = pd.DataFrame(encoded_actions, columns=[f"Feature_{i+1}" for i in range(encoding_dim)])
encoded_df["action_type"] = df["action_type"]

In [18]:
# Show results
print(encoded_df.head())


   Feature_1  Feature_2  Feature_3  Feature_4  Feature_5  action_type
0   1.457295   1.210378   1.633808   1.942718   1.519946            0
1   0.000019   2.144242   1.652380   0.000127   0.000386           21
2   1.457295   1.210378   1.633808   1.942718   1.519946            0
3   1.457295   1.210378   1.633808   1.942718   1.519946            0
4   0.000019   2.144242   1.652380   0.000127   0.000386           21
